In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

from mediapipe.python.solutions.hands import HandLandmark

In [ ]:
mp_drawing = mp.solutions.drawing_utils # easier to render
mp_hands = mp.solutions.hands # hand model

In [ ]:
os.mkdir('../Output Images')

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8 , min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read() # frame is the image
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # recolour the image BGR -> RGB
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        print(results)
        
        # rendering results
        if results.multi_hand_landmarks:  # check if anything in there
            for num, hand in enumerate(results.multi_hand_landmarks):  # loop through each result
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=4, circle_radius=4),
                                          mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=4, circle_radius=2)
                                          )

        cv2.imwrite(os.path.join('../Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        cv2.imshow('Hand tracking', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.read()
cv2.destroyAllWindows()

In [ ]:
results.multi_hand_landmark

### 4. Detect Left and Right Hands

In [ ]:
results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST]

In [ ]:
results.multi_handedness

In [ ]:
def get_label(index, hand, results):
    output = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            label = classification.classification[0].label
            score = classification.classification[0].score
            text = '{} {}'.format(label, round(score,2))
            
            # extracting coordinates
            coords = tuple(np.multiply(
                    np.array(
                        (
                         hand.landmark[mp_hands.HandLandmark.WRIST].x,
                          hand.landmark[mp_hands.HandLandmark.WRIST].y
                        ))
                [640, 480]).astype(int)) 
            output = text, coords
    return output